In [1]:
import wmfdata 

You are using Wmfdata v2.4.0, but v2.5.0 is available.

To update, run `pip install --upgrade git+https://gitlab.wikimedia.org/repos/data-engineering/wmfdata-python.git@release`.

To see the changes, refer to https://gitlab.wikimedia.org/repos/data-engineering/wmfdata-python/blob/release/CHANGELOG.md.


In [3]:
title_df = wmfdata.mariadb.run("""
SELECT 
  page_id,
  page_title
FROM page
WHERE page_namespace = 0
  AND page_is_redirect = 0;
""", dbs='enwiki')

In [4]:
title_df

,page_id,page_title
0,12,Anarchism
1,39,Albedo
2,290,A
3,303,Alabama
4,305,Achilles
...,...,...
7081676,81449111,Mohamed_Farid_Mohamed_Zawawi
7081677,81449120,Cetraria_platyphylla
7081678,81449145,Cody_Dempster
7081679,81449160,To_the_Top_(Psychic_Fever_song)


In [5]:
title_df.to_pickle('enwiki_titles_20251027.pkl')

In [ ]:
page_content_df = wmfdata.mariadb.run("""
SELECT
  p.page_id,
  p.page_namespace,
  p.page_title,
  r.rev_id     AS latest_rev_id,
  t.old_text   AS wikitext
FROM page AS p
JOIN revision AS r
  ON r.rev_id = p.page_latest
-- join the “main” slot of each revision:
JOIN slots AS s
  ON s.slot_revision_id = r.rev_id
JOIN slot_roles AS sr
  ON sr.role_id = s.slot_role_id
     AND sr.role_name = 'main'
JOIN content AS c
  ON c.content_id = s.slot_content_id
JOIN text AS t
  ON -- the content_address is generally of form 'tt:<old_id>'
     CAST(SUBSTRING(c.content_address, 4) AS UNSIGNED) = t.old_id
     AND LEFT(c.content_address, 3) = 'tt:'
WHERE
  p.page_namespace = 0            -- limit to main namespace (optional)
  AND p.page_is_redirect = 0      -- exclude redirects (optional)

LIMIT 10
""", dbs='enwiki')